# Optimizing Data on Loans

This code seeks to optimize the memory needed to store financial lending data from 2007-2011on approved loans from Lending Club (https://www.lendingclub.com/info/download-data.action).  The objective of the code is to handle the data in chunks so that maximum memory needed at any give time is 10MB in size. 

In [1]:
import pandas as pd
pd.options.display.max_columns = 99

In [2]:
print(pd.read_csv("loans_2007.csv", parse_dates=["issue_d","earliest_cr_line","last_credit_pull_d"], nrows=5))

        id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  1077501  1296599.0     5000.0       5000.0           4975.0   36 months   
1  1077430  1314167.0     2500.0       2500.0           2500.0   60 months   
2  1077175  1313524.0     2400.0       2400.0           2400.0   36 months   
3  1076863  1277178.0    10000.0      10000.0          10000.0   36 months   
4  1075358  1311748.0     3000.0       3000.0           3000.0   60 months   

  int_rate  installment grade sub_grade                 emp_title emp_length  \
0   10.65%       162.87     B        B2                       NaN  10+ years   
1   15.27%        59.83     C        C4                     Ryder   < 1 year   
2   15.96%        84.33     C        C5                       NaN  10+ years   
3   13.49%       339.31     C        C1       AIR RESOURCES BOARD  10+ years   
4   12.69%        67.79     B        B5  University Medical Group     1 year   

  home_ownership  annual_inc verification_status  

In [3]:
temp = pd.read_csv("loans_2007.csv", nrows=1000)
#Figure out a threshold of observations that fits the memory constraint
print("Memory usage (1000 observations): {}".format(round(temp.memory_usage(deep=True).sum()/1048576,2)))
#1000 rows only is equivalent to 1.55 MB
temp = pd.read_csv("loans_2007.csv", nrows=3000)
print("Memory usage (3000 observations): {}".format(round(temp.memory_usage(deep=True).sum()/1048576,2)))
#3000 rows comes out to about 4.5 MB
print(temp.shape)

Memory usage (1000 observations): 1.54
Memory usage (3000 observations): 4.62
(3000, 52)


In [4]:
#code processes the data in chunks and computes some statistics to identify how to convert some of the column data and clean it

chunk_iter = pd.read_csv("loans_2007.csv", chunksize=3000)
col_dtypes = []
col_string_unique_l50 = []
no_missing = []
memory_footprints = []
for chunk in chunk_iter:
    col_dtypes.append(chunk.dtypes.value_counts())
    #select only the string columns
    string_cols = chunk.select_dtypes(include=["object"])
    count = 0
    for s in string_cols.columns:
        share_unique = len(string_cols[s].unique())/len(string_cols[s])
        if share_unique < 0.50:
            count+=1
    #count contains the number of string columns that are less than 50% uniq
    col_string_unique_l50.append(count)
    #float columns with no missing values and are candidates for conversion
    float_cols = chunk.select_dtypes(include=["float"])
    for f in float_cols.columns:
        no_missing_cols = []
        num_missing = len(float_cols[f].index)-float_cols[f].count()
        if num_missing == 0:
            no_missing_cols.append(f)
    no_missing.append(no_missing_cols)
    memory_footprints.append(chunk.memory_usage(deep=True).sum()/1048576)   
  
print(col_string_unique_l50)
print(no_missing)
total_memory_usage = sum(memory_footprints)
print(total_memory_usage)

[20, 20, 20, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 18]
[['tax_liens'], ['tax_liens'], ['tax_liens'], ['tax_liens'], ['tax_liens'], ['tax_liens'], ['tax_liens'], ['tax_liens'], ['tax_liens'], ['tax_liens'], ['tax_liens'], ['tax_liens'], ['tax_liens'], [], []]
65.75837230682373


In [5]:
#Determine the string columns that can be converted to numeric if 
#they are cleaned up
temp = pd.read_csv("loans_2007.csv", nrows=5)
string_cols = temp.select_dtypes(include=["object"])
print(string_cols)

#Determine columns that have a few unique values and convert them to category type
category_cols = ["grade","sub_grade","emp_length","home_ownership","verification_status","loan_status","addr_state","application_type"]

memory_footprints = []
chunk_iter = pd.read_csv("loans_2007.csv", chunksize=3000)
for chunk in chunk_iter:
    chunk["int_rate"] = chunk["int_rate"].str.rstrip("%").astype(float)
    chunk["revol_util"] = chunk["revol_util"].str.rstrip("%").astype(float)
    #chunk["term"] = chunk["term"].str.rstrip(" months").astype(int)
    for c in category_cols:
        chunk[c]=chunk[c].astype("category")
    memory_footprints.append(chunk.memory_usage(deep=True).sum()/1048576)   
  
total_memory_usage = sum(memory_footprints)
print(total_memory_usage)
#able to cut memory size down from 66MB to 41MB



         term int_rate grade sub_grade                 emp_title emp_length  \
0   36 months   10.65%     B        B2                       NaN  10+ years   
1   60 months   15.27%     C        C4                     Ryder   < 1 year   
2   36 months   15.96%     C        C5                       NaN  10+ years   
3   36 months   13.49%     C        C1       AIR RESOURCES BOARD  10+ years   
4   60 months   12.69%     B        B5  University Medical Group     1 year   

  home_ownership verification_status   issue_d  loan_status pymnt_plan  \
0           RENT            Verified  Dec-2011   Fully Paid          n   
1           RENT     Source Verified  Dec-2011  Charged Off          n   
2           RENT        Not Verified  Dec-2011   Fully Paid          n   
3           RENT     Source Verified  Dec-2011   Fully Paid          n   
4           RENT     Source Verified  Dec-2011      Current          n   

          purpose                 title zip_code addr_state earliest_cr_line  \


In [6]:
#optimize numric columns
temp = pd.read_csv("loans_2007.csv", nrows=20)
float_cols = temp.select_dtypes(include=["float"])
#print(float_cols)

#float columns that contain missing values
#float columns that we can convert to more space efficient sub_type
float_int_cols = ["member_id","loan_amnt","funded_amnt", "funded_amnt_inv","annual_inc","delinq_2yrs","inq_last_6mths","open_acc","pub_rec","revol_bal","total_acc","collections_12_mths_ex_med","policy_code","acc_now_delinq"]

#Determine columns that have a few unique values and convert them to category type
category_cols = ["grade","sub_grade","emp_length","home_ownership","verification_status","loan_status","addr_state","application_type"]

memory_footprints = []
chunk_iter = pd.read_csv("loans_2007.csv", chunksize=3000)
for chunk in chunk_iter:
    chunk["int_rate"] = chunk["int_rate"].str.rstrip("%").astype(float)
    chunk["revol_util"] = chunk["revol_util"].str.rstrip("%").astype(float)
    #chunk["term"] = chunk["term"].str.rstrip(" months").astype(int)
    for c in category_cols:
        chunk[c]=chunk[c].astype("category")
    for f in float_int_cols:
        chunk[f]=pd.to_numeric(chunk[f],downcast="integer")
    memory_footprints.append(chunk.memory_usage(deep=True).sum()/1048576)   
  
#Space savings here is relatively minor by downcasting floats to integ
total_memory_usage = sum(memory_footprints)
print("Memory usage: {}".format(round(total_memory_usage,2)))


Memory usage: 38.21


In [21]:
import pandas as pd
import numpy as np
import math

#function to check if an item is a percentage
def is_percentage(item):
    if str(item).endswith("%") | str(item).endswith(" months"):
        try:
            float(item[:-1])
            return True
        except ValueError:
            return False
    else:
        return False

#function to check if an item is a number
def is_number(item):
    try:
        float(item)
        return True
    except ValueError:
        return False

#Function to automate the data compression tasks
def compress_chunk(chunk):
    #check to make sure that id is valid otherwise exceptions are thrown in the data and storage is not efficient (occurs at tail of data)
    id_is_valid = chunk.apply(lambda item: is_number(item[0]), axis=1)
    #print("Problem w/selection")
    chunk = chunk.loc[id_is_valid == True]
    #select string types
    string_cols = chunk.select_dtypes(include=["object"])
    for s in string_cols.columns:
        chunk[s] = chunk[s].str.rstrip()
        #check if the column can be mostly considered a percentage or number (thresshold 0.95)
        col_is_percentage = chunk.apply(lambda item: is_percentage(item[s]), axis=1)
        col_is_number = chunk.apply(lambda item: is_number(item[s]), axis=1)
        if col_is_percentage.sum()/len(col_is_percentage) > 0.95:
            #print("Change to float: ", s)
            chunk[s] = chunk[s].str.rstrip("%")
            chunk[s] = chunk[s].str.rstrip(" months")
            try:
                chunk[s] = chunk[s].astype(float)
            except ValueError:
                print(chunk[col_is_percentage==False])
                chunk[s] = chunk[s].astype(float,errors="ignore")
        elif col_is_number.sum()/len(col_is_number) > 0.95:
            #print(s)
            #drop rows that do not conform to the number requirements so errors are not thrown
            chunk = chunk[col_is_number==True]
        else:
            chunk[s]=chunk[s].fillna("")
            share_unique = len(string_cols[s].unique())/len(string_cols[s])
            #Make sure that these columns are not subject issues due to case values (put everything into upper case)
            chunk[s] = [st.upper() for st in chunk[s].astype(str)]
            if share_unique < 0.50:
                #if a small share is unique and the largest value count is the majority then drop this column from data
                cnt_large_value = chunk[s].value_counts()[0]
                if cnt_large_value/len(chunk[s]) >= 0.99:
                    chunk = chunk.drop(s, 1)
                else:
                    #just change to a category column otherwise
                    #print("Change to category: ", s)
                    chunk[s] = chunk[s].astype("category")
                    #print("error here")
    #want to check for float cols that are really integers
    #that is floor(col) = round(col)
    #dataset['deff'] = np.where(dataset['2016-11'] >= dataset['2016-12'], 0,1)
    float_cols = chunk.select_dtypes(include=["float"])
    float_int_cols = []
    for f in float_cols.columns:
        #Also check if there is little variation in the float columns and drop these columns
        if float_cols[f].std() < 0.0001:
            #print("Dropping", f)
            chunk = chunk.drop(f, 1)
        else:
            temp_floor = np.floor(chunk[f]*10)
            temp_round = np.round(chunk[f]*10,0)
            cnt_diff = np.where(temp_floor==temp_round,0,1).sum()
            #if seems like everything is an integer
            if cnt_diff == 0:
                #print("Downcasting", f)
                float_int_cols.append(f)
    for f in float_int_cols:
        chunk[f]=pd.to_numeric(chunk[f],downcast="integer")
    #Drop columns that have too many missing values (greater than 50%)
    missing = chunk.isnull().sum()
    for key,val in missing.items():
        if missing[key]/len(chunk) > 0.50:
            chunk = chunk.drop(key, 1)
    return chunk

# Function to determine optimal chunk size based on memory constraint
def optimal_chunk(file, maxmb, encodeval):
    mem = 0
    numrows = 0
    while mem < maxmb:
        numrows += 500
        temp = pd.read_csv(file, nrows=numrows, encoding=encodeval)
        mem = temp.memory_usage(deep=True).sum()/1048576
    return numrows-500

def compress_data(file,maxmb,dropcols,encodeval,parsedatecols):
    memory_footprints_full = []
    memory_footprints = []
    
    #obtain optimal chunk size
    opt_chunk = optimal_chunk(file, maxmb, encodeval)
    print("Optimal chunk for {}MB: {}".format(maxmb, opt_chunk))
    
    # Get the memory usage needed prior to compression
    chunk_iter = pd.read_csv(file, chunksize=opt_chunk, encoding=encodeval, parse_dates=parsedatecols)
    for chunk in chunk_iter:
        memory_footprints_full.append(chunk.memory_usage(deep=True).sum()/1048576)
    total_memory_usage = sum(memory_footprints_full)
    print("Memory usage (prior to compression): {}".format(round(total_memory_usage,2)))
    
    keep_cols = []
    temp = pd.read_csv(file, nrows=1, encoding=encodeval, parse_dates=parsedatecols)
    print("Number of columns in original file: {}".format(len(temp.columns)))
    for c in temp.columns:
        if c not in dropcols:
            keep_cols.append(c)
            
    # Create synthetic data that will help us identify crucial columns to keep so we do not have to read into data frame
    temp = pd.read_csv(file, nrows=opt_chunk, usecols=keep_cols, encoding=encodeval, parse_dates=parsedatecols)
    temp = compress_chunk(temp)
    print("Number of columns to keep: {}".format(len(temp.columns)))
    print(temp.columns)
    newdata = []
    # use the optimal chunk size and columns to keep to read in the chunks
    chunk_iter = pd.read_csv(file, usecols=temp.columns, chunksize=opt_chunk, encoding=encodeval, parse_dates=parsedatecols)
    for chunk in chunk_iter:
        chunk = compress_chunk(chunk)       
        memory_footprints.append(chunk.memory_usage(deep=True).sum()/1048576)
        #merge data into new set
        newdata.append(chunk)
        #try:
        #    newdata=pd.concat([newdata, chunk], axis=0)
        #except NameError:
        #    newdata=chunk
    total_memory_usage = sum(memory_footprints)
    print("Memory usage (after compression): {}".format(round(total_memory_usage,2)))
    #print("Memory usage (newdata): {}".format(round(newdata.memory_usage(deep=True).sum()/1048576,2)))
    return newdata

In [22]:
#we want data that is less than 5 MB per chunk (each compressed data chunk is stored in the list compressed data)
compressed_data = compress_data("loans_2007.csv", 5, dropcols=[], encodeval="ISO-8859-1", 
                                parsedatecols=["issue_d","earliest_cr_line","last_pymnt_d","last_credit_pull_d"])

Optimal chunk for 5MB: 3000
Memory usage (prior to compression): 56.51
Number of columns in original file: 52
Number of columns to keep: 43
Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'purpose', 'title', 'zip_code', 'addr_state',
       'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'last_credit_pull_d', 'pub_rec_bankruptcies'],
      dtype='object')


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Memory usage (after compression): 14.7


In [23]:
print(compressed_data[0].head(5))
print(compressed_data[0].dtypes)

        id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  1077501    1296599       5000         5000           4975.0   36 MONTHS   
1  1077430    1314167       2500         2500           2500.0   60 MONTHS   
2  1077175    1313524       2400         2400           2400.0   36 MONTHS   
3  1076863    1277178      10000        10000          10000.0   36 MONTHS   
4  1075358    1311748       3000         3000           3000.0   60 MONTHS   

   int_rate  installment grade sub_grade                 emp_title emp_length  \
0     10.65       162.87     B        B2                            10+ YEARS   
1     15.27        59.83     C        C4                     RYDER   < 1 YEAR   
2     15.96        84.33     C        C5                            10+ YEARS   
3     13.49       339.31     C        C1       AIR RESOURCES BOARD  10+ YEARS   
4     12.69        67.79     B        B5  UNIVERSITY MEDICAL GROUP     1 YEAR   

  home_ownership  annual_inc verification_st

## Conclusion

Data compression can significantly increase the efficiency and usability of data.  Significant gains were observed in converting from string to integers, string to category, and float to integers.  The functions that were created above makes it easy to determine the optimal chunk size to read in given a certain level of memory constraints and apply a similar process to other file types in order to compress the data.  We found significant savings in memory going from 66MB to 14MB.
